# Entrenamiento de Modelos de Detección de Corrosión con YOLO

Este notebook contiene el proceso completo para entrenar modelos de detección de corrosión utilizando YOLO. Las principales etapas son:

1. **Instalación e importación de librerías**: Se instalan y cargan todas las dependencias necesarias.
2. **Obtención y preparación del dataset**: Se descarga un dataset de corrosión desde Roboflow, que consta de 5 versiones, y se fusionan en un único conjunto de datos.
3. **Unificación de clases**: Se procesan los archivos de etiquetas para unificar todas las clases en una sola categoría: `corrosion`.
4. **Entrenamiento de modelos**: Se entrenan dos arquitecturas de YOLO (Nano y Large) con el dataset preparado.
5. **Guardado de modelos**: Los mejores pesos (`best.pt`) de cada entrenamiento se guardan en una carpeta dedicada (`modelos_entrenados`).
6. **Visualización de métricas**: Se muestran las gráficas de rendimiento y las matrices de confusión para cada modelo entrenado.

## 1. Instalación e Importación de Librerías

In [1]:
!pip install ultralytics roboflow

import os
import shutil
import glob
import torch
import cv2
import matplotlib.pyplot as plt
from roboflow import Roboflow
from ultralytics import YOLO
from IPython.display import display

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 31.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 783.6/783.6 kB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 61.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 38.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 63.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 83.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB

## 2. Obtención y Preparación del Dataset

Se descarga el dataset de corrosión desde Roboflow. Este dataset está compuesto por 5 versiones diferentes que se fusionarán en un único conjunto de datos para el entrenamiento. 

Para la descarga es necesaria una API Key de Roboflow. Puedes encontrarla en la documentación oficial: https://docs.roboflow.com/api-reference/authentication

In [2]:
rf = Roboflow(api_key="jkNjmBUggWEmVWag4qo0") # Reemplaza con tu API Key
project = rf.workspace("universitas-diponegoro-rfxta").project("corrosion-hsmae")

# Ruta raíz para el dataset combinado
merged_root = "dataset_corrosion_merged"

# Se crean los subdirectorios para el dataset fusionado
splits = ["train", "valid", "test"]
for split in splits:
    os.makedirs(os.path.join(merged_root, "images", split), exist_ok=True)
    os.makedirs(os.path.join(merged_root, "labels", split), exist_ok=True)

image_counter = {"train": 0, "valid": 0, "test": 0}

# Se recorren las 5 versiones del dataset, se descargan y se fusionan
for version_num in range(1, 6):
    print(f"\nDescargando y procesando versión {version_num}...")
    version = project.version(version_num)
    dataset = version.download("yolov12")

    for split in splits:
        split_images_path = os.path.join(dataset.location, split, "images")
        split_labels_path = os.path.join(dataset.location, split, "labels")

        if os.path.exists(split_images_path) and os.path.exists(split_labels_path):
            for filename in os.listdir(split_images_path):
                # Se copia la imagen
                src_img = os.path.join(split_images_path, filename)
                dst_img = os.path.join(merged_root, "images", split, f"{image_counter[split]:06d}.jpg")
                shutil.copyfile(src_img, dst_img)

                # Se copia la etiqueta correspondiente
                label_name = filename.replace(".jpg", ".txt").replace(".png", ".txt")
                src_lbl = os.path.join(split_labels_path, label_name)
                dst_lbl = os.path.join(merged_root, "labels", split, f"{image_counter[split]:06d}.txt")

                if os.path.exists(src_lbl):
                    shutil.copyfile(src_lbl, dst_lbl)

                image_counter[split] += 1

print("\n¡Dataset completamente fusionado!")

# Se crea el archivo data.yaml necesario para el entrenamiento de YOLO
yaml_path = os.path.join(merged_root, "data.yaml")
with open(yaml_path, "w") as f:
    f.write(
        "train: images/train\n"
        "val: images/valid\n"
        "test: images/test\n"
        "\n"
        "nc: 1\n"
        "names: ['corrosion']\n"
    )

print(f"Archivo data.yaml creado en: {yaml_path}")

loading Roboflow workspace...
loading Roboflow project...

Descargando y procesando versión 1...



Extracting Dataset Version Zip to corrosion-1 in yolov12:: 100%|██████████| 2142/2142 [00:00<00:00, 13430.46it/s]



Descargando y procesando versión 2...


KeyboardInterrupt: 

## 3. Unificación de Clases de Corrosión

El dataset original contiene múltiples clases para la corrosión (ej. `korosi`, `oxido`). Para simplificar el problema, se unificarán todas estas clases en una única clase `corrosion` (índice 0).

In [3]:
def force_first_char_zero(labels_root):
    """
    Recorre todos los archivos .txt en los subdirectorios (train, valid, test)
    y cambia el primer carácter de cada línea a '0' para unificar la clase.
    """
    for split in ("train", "valid", "test"):
        folder = os.path.join(labels_root, split)
        for path in glob.glob(f"{folder}/*.txt"):
            with open(path, "r") as f:
                lines = f.read().splitlines()
            
            new_lines = []
            for line in lines:
                if not line.strip():
                    continue
                new_lines.append("0" + line[1:] + "\n")
            
            with open(path, "w") as f:
                f.writelines(new_lines)

labels_root = "dataset_corrosion_merged/labels"
force_first_char_zero(labels_root)
print("Todas las clases han sido unificadas a '0' en los archivos de etiquetas.")

Todas las clases han sido unificadas a '0' en los archivos de etiquetas.


## 4. Entrenamiento de Modelos

Se entrenarán dos modelos con diferentes arquitecturas para comparar su rendimiento:
- **YOLOv12 Nano**: Un modelo ligero y rápido, ideal para aplicaciones en tiempo real.
- **YOLOv12 Medium**: Un modelo ligero y rápido, ideal para aplicaciones en tiempo real.
- **YOLOv12 Large**: Un modelo más grande y preciso, a costa de una mayor carga computacional.

In [5]:
# --- Variables y Funciones Auxiliares ---

# Ruta del archivo de configuración del dataset
dataset_path = './dataset_aereo'
yaml_path = f'{dataset_path}/data.yaml'
assert os.path.exists(yaml_path), f"No se encontró data.yaml en {yaml_path}"

# Directorio para guardar los modelos entrenados
output_models_dir = "modelos_entrenados"
os.makedirs(output_models_dir, exist_ok=True)

# Función para limpiar la memoria de la GPU
def clear_gpu_cache():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    print("Caché de la GPU limpiada.")

# Función para visualizar las métricas del entrenamiento
def show_training_metrics(training_path):
    """Muestra las imágenes de resultados y matriz de confusión."""
    results_img_path = os.path.join(training_path, 'results.png')
    confusion_matrix_path = os.path.join(training_path, 'confusion_matrix.png')

    if os.path.exists(results_img_path):
        img_results = cv2.imread(results_img_path)
        img_results = cv2.cvtColor(img_results, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(12, 8))
        plt.imshow(img_results)
        plt.title("Métricas de Entrenamiento")
        plt.axis('off')
        plt.show()
    else:
        print(f"No se encontró la imagen de resultados en {results_img_path}")

    if os.path.exists(confusion_matrix_path):
        img_cm = cv2.imread(confusion_matrix_path)
        img_cm = cv2.cvtColor(img_cm, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(12, 8))
        plt.imshow(img_cm)
        plt.title("Matriz de Confusión")
        plt.axis('off')
        plt.show()
    else:
        print(f"No se encontró la matriz de confusión en {confusion_matrix_path}")

### 4.1. Entrenamiento del Modelo YOLOv12 Nano

In [ ]:
clear_gpu_cache()

# Cargar el modelo YOLOv12 Nano pre-entrenado
model_n = YOLO("yolo12n.pt")

# Entrenar el modelo
results_n = model_n.train(
    data=yaml_path,
    epochs=50,
    imgsz=640,
    plots=True,
    batch=32,
    name='yolo12n_corrosion' # Nombre del directorio de resultados
)

# Guardar el mejor modelo con el nombre deseado
best_model_path_n = os.path.join(results_n.save_dir, 'weights', 'best.pt')
final_model_name_n = 'modelo-aereo-n.pt'
destination_path_n = os.path.join(output_models_dir, final_model_name_n)

if os.path.exists(best_model_path_n):
    shutil.copy(best_model_path_n, destination_path_n)
    print(f"Modelo Nano guardado en: {destination_path_n}")
else:
    print(f"Error: No se encontró el archivo 'best.pt' en {os.path.join(results_n.save_dir, 'weights')}")

Caché de la GPU limpiada.
Ultralytics 8.3.228 🚀 Python-3.9.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24215MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./dataset_aereo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo12n_corrosion2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True,

### Visualización de Métricas (YOLOv12 Nano)

In [ ]:
show_training_metrics(results_n.save_dir)

### 4.2. Entrenamiento del Modelo YOLOv12 Medium

In [7]:
clear_gpu_cache()

# Cargar el modelo YOLOv12 Large pre-entrenado
model_l = YOLO("yolo12m.pt")

# Entrenar el modelo
results_l = model_l.train(
    data=yaml_path,
    epochs=50,
    imgsz=640,
    plots=True,
    batch=16, # Se reduce el batch size por el mayor tamaño del modelo
    name='yolo12m_corrosion' # Nombre del directorio de resultados
)

# Guardar el mejor modelo con el nombre deseado
best_model_path_l = os.path.join(results_l.save_dir, 'weights', 'best.pt')
final_model_name_l = 'modelo-aereo-m.pt'
destination_path_l = os.path.join(output_models_dir, final_model_name_l)

if os.path.exists(best_model_path_l):
    shutil.copy(best_model_path_l, destination_path_l)
    print(f"Modelo Large guardado en: {destination_path_l}")
else:
    print(f"Error: No se encontró el archivo 'best.pt' en {os.path.join(results_l.save_dir, 'weights')}")

Caché de la GPU limpiada.
New https://pypi.org/project/ultralytics/8.3.232 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.228 🚀 Python-3.9.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24215MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./dataset_aereo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name

AcceleratorError: CUDA error: invalid argument
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### Visualización de Métricas (YOLOv12 Medium)

In [ ]:
show_training_metrics(results_l.save_dir)

### 4.3. Entrenamiento del Modelo YOLOv12 Large

In [ ]:
clear_gpu_cache()

# Cargar el modelo YOLOv12 Large pre-entrenado
model_l = YOLO("yolo12l.pt")

# Entrenar el modelo
results_l = model_l.train(
    data=yaml_path,
    epochs=50,
    imgsz=640,
    plots=True,
    batch=16, # Se reduce el batch size por el mayor tamaño del modelo
    name='yolo12l_corrosion' # Nombre del directorio de resultados
)

# Guardar el mejor modelo con el nombre deseado
best_model_path_l = os.path.join(results_l.save_dir, 'weights', 'best.pt')
final_model_name_l = 'modelo-aereo-l.pt'
destination_path_l = os.path.join(output_models_dir, final_model_name_l)

if os.path.exists(best_model_path_l):
    shutil.copy(best_model_path_l, destination_path_l)
    print(f"Modelo Large guardado en: {destination_path_l}")
else:
    print(f"Error: No se encontró el archivo 'best.pt' en {os.path.join(results_l.save_dir, 'weights')}")

### Visualización de Métricas (YOLOv12 Large)

In [ ]:
show_training_metrics(results_l.save_dir)